# GET DATA

In [1]:
import requests
import urllib
from bs4 import BeautifulSoup

import json
import pandas as pd

import matplotlib.pyplot as plt

from datetime import date
today = date.today()

In [2]:
# //- // Company Specific code

### Set variables for company to scrape data for company of interest
# to identify cik and proper name for company,
# search at https://www.sec.gov/edgar/searchedgar/cik.htm

# to identify component of link to 10k (annual reports), 
# search folders at https://www.sec.gov/Archives/edgar/data/cik
# where cik shall be replaced by cik of company under analysis

############ Set variables for company of interest ##
company = 'Tesla, Inc.'                             #
cik = '1318605'                                     #
link_to_10k = '/000156459021004599/'                # 
#####################################################

In [3]:
### Scraping data from SEC EDGAR 

# google for headers for your browser
headers = {'User-Agent': 'your headers go here'}
# be nice to host servers, set timeout
timeout = 3

# first part of url to scrape from - same for all companies
base_url = "https://www.sec.gov/Archives/edgar/data/"

# unique to company url with link to 10-K filling
doc_url = base_url + cik + link_to_10k + 'index.json'

# financial (10-K/ 10-Q) reports contained in FilingSummary.xml 
# write function to extract this summary from url
def get_xml_summary(url):
    
    # required to construct url to xml_summary
    home_url = r"https://www.sec.gov" 
    
    # get url content and as json file 
    content = requests.get(url, headers=headers, timeout=timeout).json()
    
    # content result is nested dictionary
    # print(content)
    for file in content['directory']['item']:
        if file['name']=='FilingSummary.xml':
            xml_summary = home_url + content['directory']['name'] + '/' + file['name']
    return xml_summary

# now apply function to extract xml_summary from url
xml_summary = get_xml_summary(doc_url)

# reports in xml_summary are stored under tag name 'myreports'
# let's get to them by parsing xml_summary 
# write function to parse xml and extract list of reports
def get_reports(xml_summary):
    
    content = requests.get(xml_summary, headers=headers, timeout=timeout).content
    soup = BeautifulSoup(content, 'lxml')
    reports = soup.find('myreports')
    #print(reports)    
    
    # store parsed reports in reports list
    fin_reports = []
   
    # each report contains several elements
    # create dictionary to store elements
    for report in reports.find_all('report')[:-1]: # last report is incomplete and causes error
        report_dict = {}
        report_dict['long_name'] = report.longname.text
        report_dict['short_name'] = report.shortname.text
        report_dict['category'] = report.menucategory.text
        report_dict['position'] = report.position.text
        report_dict['url'] = xml_summary.replace('FilingSummary.xml', '') + report.htmlfilename.text
        #print(report_dict)   
        
        # append to list of reports
        fin_reports.append(report_dict)
    return fin_reports

# apply function to extract list of reports for the company
reports_list = get_reports(xml_summary)

# print list of reports
print('-'*100)
print(f'The following statements are available from 10-K fillings by {company}')
print('')
for i in range(len(reports_list)):
    print(f"Statement position: {reports_list[i]['position']}\n{reports_list[i]['short_name']}\n{reports_list[i]['url']}")
    print('')
print('-'*100)

----------------------------------------------------------------------------------------------------
The following statements are available from 10-K fillings by Tesla, Inc.

Statement position: 1
Document and Entity Information
https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/R1.htm

Statement position: 2
Consolidated Balance Sheets
https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/R2.htm

Statement position: 3
Consolidated Balance Sheets (Parenthetical)
https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/R3.htm

Statement position: 4
Consolidated Statements of Operations
https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/R4.htm

Statement position: 5
Consolidated Statements of Operations (Parenthetical)
https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/R5.htm

Statement position: 6
Consolidated Statements of Comprehensive Income (Loss)
https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/

In [4]:
# next step is to identify statement of interest by its position 
# and parse corresponding table

# write function to parse a table
def parse_table(statement):
    
    # there are 3 parts in table
    # header, sections and data
    # will parse rows considering this difference
    
    statement_data = {}
    statement_data['headers'] = []
    statement_data['sections'] =[]
    statement_data['data'] = []
    
    # get and parse table from statement url
    content = requests.get(statement, headers=headers, timeout=timeout).content
    soup = BeautifulSoup(content, 'html')
    
    #print(soup.table.find_all('tr'))
    
    # find all rows in table
    for index, row in enumerate(soup.table.find_all('tr')):
        #print(index, row)
        
        # elements are contained in 'td' tags
        cols = row.find_all('td')
        
        # header row has 'th' tag
        if (len(row.find_all('th'))!=0):
            header_row = [ele.text.strip() for ele in row.find_all('th')]
            statement_data['headers'].append(header_row)
            
        # section rows have 'strong' tag
        elif (len(row.find_all('th'))==0 and len(row.find_all('strong'))!=0):
            sec_row = cols[0].text.strip()
            statement_data['sections'].append(sec_row)
        
        # data rows
        elif (len(row.find_all('th'))==0 and len(row.find_all('strong'))==0):
            reg_row = [ele.text.strip() for ele in cols]
            statement_data['data'].append(reg_row)
        
        # else is error
        else:
            print('Error when parsing')
        
    return statement_data

**Parse statements from scraped links**

In [5]:
### following are key statements, several more can be parsed from reports_list
st_of_financial_position = parse_table(reports_list[1]['url'])
st_of_profit_or_loss = parse_table(reports_list[3]['url'])
st_of_comprehensive_income_loss = parse_table(reports_list[5]['url'])
st_of_changes_in_equity = parse_table(reports_list[6]['url'])
st_of_cash_flows = parse_table(reports_list[8]['url'])

# following tables will require different approch when converting to dataframe
ppe = parse_table(reports_list[16]['url'])
segment_revenues = parse_table(reports_list[126]['url'])

# PARSE and CLEAN DATA

**Statement of Financial Position**

In [6]:
### STATEMENT OF FINANCIAL POSITION ###
### original table can be reviwed by running (reports_list[1]['url'])

st_of_financial_position_df = pd.DataFrame(data = st_of_financial_position['data'])

### drop empty column     
st_of_financial_position_df = st_of_financial_position_df.drop(1, axis=1)

### drop last three rows with None values
st_of_financial_position_df.drop(st_of_financial_position_df.tail(3).index, inplace=True)

### set titles of line items as index
st_of_financial_position_df.index = st_of_financial_position_df[0]
st_of_financial_position_df = st_of_financial_position_df.drop(0, axis=1)
st_of_financial_position_df.index.name = 'Balance Sheet, MLN USD'

### assign column names
st_of_financial_position_df.columns = st_of_financial_position['headers'][0][1:3]

### clean data
st_of_financial_position_df = (st_of_financial_position_df.replace('[\$, ]', '', regex=True)
                               .replace('[(]', '-', regex=True)
                               .replace('[)]', '', regex=True)
                               .replace('', 0, regex=True))

### convert values to float format
st_of_financial_position_df = st_of_financial_position_df.astype(float)

In [7]:
# //- // Company Specific code
#st_of_financial_position_df.to_csv("tesla_bs_2020.csv")

**Statement of Profit or Loss**

In [11]:
### STATEMENT OF PROFIT OR LOSS ###
### original table can be reviwed by running (reports_list[3]['url'])

st_of_profit_or_loss_df = pd.DataFrame(data = st_of_profit_or_loss['data'])

### drop empty column     
st_of_profit_or_loss_df = st_of_profit_or_loss_df.drop(1, axis=1)

### drop last three rows with None values
st_of_profit_or_loss_df.drop(st_of_profit_or_loss_df.tail(3).index, inplace=True)

### set titles of line items as index
st_of_profit_or_loss_df.index = st_of_profit_or_loss_df[0]
st_of_profit_or_loss_df = st_of_profit_or_loss_df.drop(0, axis=1)
st_of_profit_or_loss_df.index.name = 'Income Statement, MLN USD'

### assign column names
st_of_profit_or_loss_df.columns = st_of_profit_or_loss['headers'][1]

### clean data
st_of_profit_or_loss_df = (st_of_profit_or_loss_df.replace('[\$, ]', '', regex=True)
                               .replace('[(]', '-', regex=True)
                               .replace('[)]', '', regex=True)
                               .replace('', 0, regex=True))

### convert values to float format
st_of_profit_or_loss_df = st_of_profit_or_loss_df.astype(float)

In [13]:
# //- // Company Specific code
#st_of_profit_or_loss_df.to_csv("tesla_is_2020.csv")

**Statement of comprehensive income (loss)**

In [18]:
### COMPREHENSIVE INCOME (LOSS) ###

st_of_comprehensive_income_loss_df = pd.DataFrame(data = st_of_comprehensive_income_loss['data'])


### set titles of line items as index
st_of_comprehensive_income_loss_df.index = st_of_comprehensive_income_loss_df[0]
st_of_comprehensive_income_loss_df = st_of_comprehensive_income_loss_df.drop(0, axis=1)
st_of_comprehensive_income_loss_df.index.name = 'Compr. Income Statement, MLN USD'

### assign column names
st_of_comprehensive_income_loss_df.columns = st_of_comprehensive_income_loss['headers'][1]

### clean data
st_of_comprehensive_income_loss_df = (st_of_comprehensive_income_loss_df.replace('[\$, ]', '', regex=True)
                               .replace('[(]', '-', regex=True)
                               .replace('[)]', '', regex=True)
                               .replace('', 0, regex=True))

### convert values to float format
st_of_comprehensive_income_loss_df = st_of_comprehensive_income_loss_df.astype(float)

In [19]:
# //- // Company Specific code
#st_of_comprehensive_income_loss_df.to_csv("tesla__comp_is_2020.csv")

**Statement of Cash Flows**

In [23]:
### STATEMENT OF CASH FLOWS ###

st_of_cash_flows_df = pd.DataFrame(data = st_of_cash_flows['data'])

### set titles of line items as index
st_of_cash_flows_df.index = st_of_cash_flows_df[0]
st_of_cash_flows_df = st_of_cash_flows_df.drop(0, axis=1)
st_of_cash_flows_df.index.name = 'Cash Flows, MLN USD'

### assign column names
st_of_cash_flows_df.columns = st_of_cash_flows['headers'][1]

### clean data
st_of_cash_flows_df = (st_of_cash_flows_df.replace('[\$, ]', '', regex=True)
                               .replace('[(]', '-', regex=True)
                               .replace('[)]', '', regex=True)
                               .replace('', 0, regex=True))

### convert values to float format
st_of_cash_flows_df = st_of_cash_flows_df.astype(float)

In [24]:
# //- // Company Specific code
#st_of_cash_flows_df.to_csv("tesla_cf_2020.csv")

**Statement of changes in Shareholders' Equity**

In [25]:
### Statement of changes in shareholders' equity
st_of_changes_in_equity_df = pd.DataFrame(st_of_changes_in_equity['data'])

### drop empty column     
st_of_changes_in_equity_df = st_of_changes_in_equity_df.drop([1, 7, 9], axis=1)

### drop last three rows with None values
st_of_changes_in_equity_df.drop(st_of_changes_in_equity_df.tail(3).index, inplace=True)

### set titles of line items as index
st_of_changes_in_equity_df.index = st_of_changes_in_equity_df[0]
st_of_changes_in_equity_df = st_of_changes_in_equity_df.drop(0, axis=1)
st_of_changes_in_equity_df.index.name = 'Statement of Equity'


### assign column names
hd = st_of_changes_in_equity['headers'][0][1:].copy()
del hd[7]
del hd[8]
del hd[9]
st_of_changes_in_equity_df.columns = hd

### clean data
st_of_changes_in_equity_df = (st_of_changes_in_equity_df.replace('[\$, ]', '', regex=True)
                               .replace('[(]', '-', regex=True)
                               .replace('[)]', '', regex=True)
                               .replace('', 0, regex=True))

### convert values to float format
st_of_changes_in_equity_df = st_of_changes_in_equity_df.astype(float)

In [26]:
# //- // Company Specific code
#st_of_changes_in_equity_df.to_csv("tesla_equity_2020.csv")

**Property, Plant & Equipment**

In [27]:
### set variables for years in reverse order year_1 being the latest
year_1 = st_of_profit_or_loss_df.columns[0]
year_2 = st_of_profit_or_loss_df.columns[1]
year_3 = st_of_profit_or_loss_df.columns[2]

### parsed list contains table, rows at list position 2-12
def fun(item):
    to_remove = ['', '$']
    if (item not in to_remove):
        return True
    else:
        return False
    
frames = []
for i in range(3,12):
    frames.append(pd.DataFrame([parse_table(reports_list[16]['url'])['data'][i]]))
ppe_df = pd.concat(frames)

### set titles of line items as index
ppe_df.index = ppe_df[ppe_df.columns[0]]
ppe_df = ppe_df.drop(0, axis=1)
ppe_df.index.name = 'Property, Plant & Eq, MLN USD'

### clean data
ppe_df = (ppe_df.replace('[\$, ]', '', regex=True)
          .replace('[(]', '-', regex=True)
          .replace('[)]', '', regex=True)
                               .replace('', 0, regex=True))

### convert values to float format
ppe_df = ppe_df.astype(float)

### drop columns with zero values
ppe_df = ppe_df.drop([1,2,4,5,6,8], axis=1)

### assign headings
ppe_df.columns = [year_1, year_2]

In [28]:
# //- // Company Specific code
#ppe_df.to_csv("tesla_ppe_2020.csv")

**Revenue by Geography**

In [35]:
### Revenue by geography

frames = []
for i in range(7):
    frames.append(pd.DataFrame([segment_revenues['data'][i][-3:]]))

segments_revenues_df = pd.concat(frames)

### clean data
segments_revenues_df = (segments_revenues_df.replace('[\$, ]', '', regex=True)
                        .replace('[(]', '-', regex=True)
                        .replace('[)]', '', regex=True)
                        .replace('', 0, regex=True))

### convert values to float
segments_revenues_df = segments_revenues_df.astype('float')

### drop columns with all 'zeros'
segments_revenues_df = segments_revenues_df.loc[(segments_revenues_df!=0).any(1)]


### assign headings
index_cols = []
for i in range(7):
    index_cols.append(segment_revenues['data'][i][0])
    
index_cols = list(dict.fromkeys(index_cols))

segments_revenues_df.index = index_cols
segments_revenues_df.index.name = 'Segment Revenue, MLN USD'
segments_revenues_df.columns = segment_revenues['headers'][1][-3:]

In [37]:
# //- // Company Specific code
#segments_revenues_df.to_csv("tesla_segments_2020.csv")